In [ ]:
import requests
import urllib
from bs4 import BeautifulSoup
import random
import time
from tqdm import tqdm_notebook as tqdm
import re
import unicodedata
import collections
import re
import string
import json
import pandas as pd

import gensim
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

from pymystem3 import Mystem
mystem = Mystem()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Стихи, написанные человеком

#### www.stihi.ru

+ Актуальные стихи

In [ ]:
url = 'https://www.stihi.ru/poems/list.html?day=1&month=03&year=2019&topic=all&type=selected&start='

def get_links(url):
    links = list()
    for x in tqdm(range(1, 2000, 10)):
        req = requests.get(url + str(x))
        soup = BeautifulSoup(req.text, "lxml")
        for poetry in soup.find_all('a', attrs={'class':'poemlink'}):
            links.append('https://www.stihi.ru' + str(poetry.get('href')))
 
    return links

In [ ]:
links = get_links(url)

+ Актуальные авторы

In [ ]:
url = 'https://www.stihi.ru/authors/editor.html'
req = requests.get(url)
links_auth = list()
soup = BeautifulSoup(req.text, "lxml")
for author in soup.find_all('a', attrs={'class':'recomlink'}):
    links_auth.append('https://www.stihi.ru/' + str(author.get('href')))  

for link in tqdm(links_auth):
    req = requests.get(link)
    soup = BeautifulSoup(req.text, "lxml")
    for poetry in soup.find_all('a', attrs={'class':'poemlink'}):
        links.append('https://www.stihi.ru' + str(poetry.get('href')))  

+ Другие

In [ ]:
url = 'https://www.stihi.ru/poems/list.html?topic=all&start='

links_all = get_links(url)

Извлекаем текст стихотворения из html + проставляем Person в качестве автора

In [ ]:
def get_info(links):
    authors = list()
    texts = list()
    for link in tqdm(links):
        req = requests.get(link)
        soup = BeautifulSoup(req.text, "lxml")    
        for_text = soup.find_all('div', attrs={'class':'text'})
        for text in for_text:
            text = unicodedata.normalize("NFKD", text.text)
            authors.append('Person')
            texts.append(text)

    return authors, texts

In [ ]:
authors, texts = get_info(links_all)

In [ ]:
df = pd.DataFrame({'author': authors, 'poem': texts})

In [ ]:
df.to_csv('/Users/alinashaymardanova/Desktop/person_add.csv', sep='\t')

---

### Нейропоэзия

yaya.poet

autopoems

yandexautopoet

NB Срок действия ключа прошёл, поэтому могу выложить в открытый доступ

In [ ]:
token = 'access_token=0cdeb4c66798a3b9fd5a4d93c5004508f539a3f68493ca355d1e069b6b3eb4eb8b414435fac1292a18170'

In [ ]:
def get_poetry(token):
    req = urllib.request.Request('https://api.vk.com/method/wall.get?domain=yandexautopoet&'+ token + '&count=1&v=5.74')
    response = urllib.request.urlopen(req)
    result = response.read().decode('utf-8')
    data = json.loads(result)
    count = data['response']['count']
    i = 0
    step = count*0.03
    t = list()
    while i <= count:
        print(i)
        req = urllib.request.Request('https://api.vk.com/method/wall.get?domain=yandexautopoet' + '&' + token + '&' +'count=' + str(count)+'&v=5.74&offset='+str(i))
        response = urllib.request.urlopen(req)
        result = response.read().decode('utf-8')
        data = json.loads(result)
        time.sleep(3)
        t.append(data)
        i += step
    
    return t

In [ ]:
t = get_poetry(token)

Парсим текст + записываем автора

In [ ]:
# authors = list()
# poetry = list()
for el in t:
    for texts in el['response']['items']:
        authors.append('Computer')
        poetry.append(texts['text'])

In [ ]:
df = pd.DataFrame({'author': authors, 'poem': poetry})

In [ ]:
df.to_csv('/Users/alinashaymardanova/Desktop/comp.csv', sep='\t')

Кибер-Пушкина парсим отдельно

http://www.teterin.ru/pushkin/#st

In [ ]:
url = 'http://www.teterin.ru/pushkin/#st'
req = requests.get(url)
req.encoding = 'cp1251'
links_auth = list()
soup = BeautifulSoup(req.text, "lxml")
soup

In [ ]:
text = re.sub('\n     \n', ' \n\n', text).split('\n\n')

In [ ]:
authors = list()
poetry = list()
for poet in text:
    authors.append('Computer')
    poetry.append(poet)